In [1]:
from src.evaluation import foo

foo()

importing foo from module.


In [ ]:
# Kaggle API requires an access token
# https://github.com/Kaggle/kagglehub

# export KAGGLE_API_TOKEN=xxxxxxxxxxxxxx

In [9]:
from pathlib import Path
import os, kagglehub

project_root = Path.cwd()
data_dir = project_root / "data"
data_dir.mkdir(parents=True, exist_ok=True)

os.environ["KAGGLEHUB_CACHE"] = str(data_dir)

dataset_path = kagglehub.dataset_download("zlatan599/mushroom1")

100%|██████████| 11.3G/11.3G [07:24<00:00, 27.3MB/s]

Extracting files...


/workspace/data
